In [1]:
import pandas as pd
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.models import Sequential
from keras.src.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from scipy import stats
import ast


In [2]:
hp_set = [(1, [1, 0, 0]), (1, [32, 0, 0]), (2, [1, 1, 0]), (2, [1, 32, 0]), (2, [32, 1, 0]), (2, [32, 32, 0]), (3, [1, 1, 1]), (3, [1, 1, 32]), (3, [1, 32, 1]), (3, [32, 1, 1]), (3, [1, 32, 32]), (3, [32, 32, 1]), (3, [32, 1, 32]), (3, [32, 32, 32])]

In [3]:
def create_mlp(X, y, num_layers, num_neurons):
    model = Sequential()

    model.add(Dense(num_neurons[0], input_dim = X.shape[1], activation = 'relu'))

    for i in range(1, num_layers):
        model.add(Dense(num_neurons[i], activation = 'relu'))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [4]:
def make_splits(input_dir, split_dir):
    for file in os.listdir(input_dir):
        print(file)
        dataset = pd.read_csv(input_dir + '\\' + file)
        # print(dataset)
        X = dataset.iloc[:, :-1].values  # All columns except the last one for features
        y = dataset.iloc[:, -1].values   # Last column for target
        # print(labels)
        kf = KFold(n_splits=10, shuffle=True)
        for fold, (train_index, val_index) in enumerate(kf.split(X), start=1):
            dataset.iloc[train_index].to_csv(split_dir + '\\' + f'Train Fold {fold} ' + file, index=False)
            dataset.iloc[val_index].to_csv(split_dir + '\\' + f'Val Fold {fold} ' + file, index=False)

In [6]:
make_splits(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Formatted Datasets', r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Dataset Splits')

100_plants_margin.csv
100_plants_shape.csv
100_plants_texture.csv
amazon_review.csv
analcatdata_authorship.csv
AP_Colon_Kidney.csv
ar4.csv
ar6.csv
banana_quality.csv
Banknote.csv
Bioresponse.csv
breast-cancer-wisconsin.csv
cardiotocography.csv
car_data.csv
Click Prediction.csv
Climate Simulation.csv
cnae9.csv
Credit Card.csv
dataset_11_balance-scale.csv
dataset_12_mfeat-factors.csv
dataset_14_mfeat-fourier.csv
dataset_16_mfeat-karhunen.csv
dataset_185_yeast.csv
dataset_186_satimage.csv
dataset_18_mfeat-morphological.csv
dataset_22_mfeat-zernike.csv
dataset_30_page-blocks.csv
dataset_36_segment.csv
dataset_37_diabetes.csv
dataset_40_sonar.csv
dataset_44_spambase.csv
dataset_53_heart-statlog.csv
dataset_54_vehicle.csv
dataset_59_ionosphere.csv
dataset_60_waveform-5000.csv
dataset_6_letter.csv
default of credit card clients.csv
dna.csv
Dry_Bean_Dataset.csv
EEG Eye State.csv
electricity-normalized.csv
Fertility.csv
first_order_theorem_proving.csv
GesturePhaseSegmentationProcessed.csv
glass

In [6]:
def hp_space_experiment(input_dir, split_dir, results_dir, hp_set):
    results = {}
    complete_results = {}

    for file in os.listdir(input_dir):
        print(file)
        cur_result = {}
        cur_result_complete = {}
        dataset = pd.read_csv(input_dir + '\\' + file)
        # print(dataset)
        X = dataset.iloc[:, :-1].values  # All columns except the last one for features
        y = dataset.iloc[:, -1].values   # Last column for target
        # print(labels)
        kf = KFold(n_splits=10, shuffle=True)
        for fold, (train_index, val_index) in enumerate(kf.split(X), start=1):
            dataset.iloc[train_index].to_csv(split_dir + '\\' + f'Train Fold {fold} ' + file, index=False)
            dataset.iloc[val_index].to_csv(split_dir + '\\' + f'Val Fold {fold} ' + file, index=False)
        # print(train_index)
        # print(features.type)
            X_train, X_val = X[train_index], X[val_index]
            # print(train_index.dtype, val_index.dtype)
            y_train, y_val = y[train_index], y[val_index]
            for i in range(len(hp_set)):
                hp = hp_set[i]
                if i not in cur_result:
                    cur_result[i] = []
                if i not in cur_result_complete:
                    cur_result_complete[i] = []
                model = create_mlp(X_train, y_train, hp[0], hp[1])
                model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
                val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
                cur_result[i].append(val_accuracy)
                cur_result_complete[i].append(val_accuracy)
                y_pred_train = model.predict(X_train).flatten()
                y_pred_val = model.predict(X_val).flatten()
                y_pred = list(y_pred_train) + list(y_pred_val)
                y_total = list(y_train) + list(y_val)
                pred_df = pd.DataFrame({'y' : y_total, 'y_pred' : y_pred})
                # print(pred_df)
                pred_df.to_csv(results_dir + '\\' + f'Complete Results Fold {fold} {hp_set[i]} ' + file, index=False)
        
        complete_results[file] = cur_result_complete
        # print(complete_results)
        for i in cur_result:
            cur_result[i] = np.mean(cur_result[i])
        results[file] = cur_result
    
    return results, complete_results

In [5]:
def hp_space_experiment_from_split(dir, hp_set):
    input_dir = dir + '\\' + 'Formatted Datasets'
    split_dir = dir + '\\' + 'Splitted Datasets'
    results = {}
    complete_results = {}

    for file in os.listdir(input_dir):
        print(file)
        cur_result = {}
        cur_result_complete = {}
        for fold in range(1, 11):
            train_df = pd.read_csv(split_dir + '\\' + f'Train Fold {fold} ' + file)
            val_df = pd.read_csv(split_dir + '\\' + f'Val Fold {fold} ' + file)
            X_train = train_df.iloc[:, :-1].values  
            y_train = train_df.iloc[:, -1].values   
            X_val = val_df.iloc[:, :-1].values  
            y_val = val_df.iloc[:, -1].values
            for i in range(len(hp_set)):
                hp = hp_set[i]
                if i not in cur_result:
                    cur_result[i] = []
                if i not in cur_result_complete:
                    cur_result_complete[i] = []
                model = create_mlp(X_train, y_train, hp[0], hp[1])
                model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
                val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
                cur_result[i].append(val_accuracy)
                cur_result_complete[i].append(val_accuracy)
                y_pred_train = model.predict(X_train).flatten()
                y_pred_val = model.predict(X_val).flatten()
                y_pred = list(y_pred_train) + list(y_pred_val)
                y = list(y_train) + list(y_val)
                pred_df = pd.DataFrame({'y' : y, 'y_pred' : y_pred})
                # print(pred_df)
                pred_df.to_csv(dir + '\\' + 'Complete Results' + '\\' + f'Complete Results Fold {fold} {hp_set[i]} ' + file, index=False)
        
        complete_results[file] = cur_result_complete
        print(complete_results)
        for i in cur_result:
            cur_result[i] = np.mean(cur_result[i])
        results[file] = cur_result
    
    return results, complete_results

In [6]:
for i in [5]:
    print(f'Batch Valid {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Batch Valid ' + str(i)
    make_splits(dir + r'\Formatted Datasets', dir + r'\Splitted Datasets')
    results, complete_results = hp_space_experiment_from_split(dir, test_hp_set)
    results_df = pd.DataFrame(results).T
    results_complete_df = pd.DataFrame(complete_results).T
    results_df.to_csv(dir + '\\' + f'Batch Valid {i} Results.csv')
    results_complete_df.to_csv(dir + '\\' + f'Batch Valid {i} Results Complete.csv')

Batch Valid 5
gym_members_exercise_tracking.csv
Haberman.csv
HARTH_downsampled.csv
hcv.csv
Healthcare-Diabetes.csv
heart_failure_clinical_records_dataset.csv
HTRU_2.csv
Image Segmentation.csv
Indian Liver Patient Dataset (ILPD).csv
iris.csv
gym_members_exercise_tracking.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/st

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/s

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
3/3 ━━━━━━━━

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
3/3 ━━━━━━━━

In [9]:
for i in range(6, 8):
    print(f'Batch {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Batch ' + str(i)
    make_splits(dir + r'\Formatted Datasets', dir + r'\Splitted Datasets')
    results, complete_results = hp_space_experiment_from_split(dir, test_hp_set)
    results_df = pd.DataFrame(results).T
    results_complete_df = pd.DataFrame(complete_results).T
    results_df.to_csv(dir + '\\' + f'Batch {i} Results.csv')
    results_complete_df.to_csv(dir + '\\' + f'Batch {i} Results Complete.csv')

Batch 6
hiva_agnostic.csv
HTRU_2.csv
Indian Liver Patient Dataset (ILPD).csv
iris.csv
isolet.csv
JapaneseVowels.csv
jm1.csv
jungle_chess_2pcs_raw_endgame_complete.csv
kc1.csv
kc2.csv
hiva_agnostic.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
9/9 ━━━━━━━━

...

1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 1s 510us/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 541us/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 1s 693us/step
215/215 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step
1931/1931 ━━━━━━━━━━━━━━━━━━━━ 2s 755us/step
215/215 ━━━━━━━━━━━━━━━

In [ ]:
test_df = pd.DataFrame(test_result)
test_df_complete = pd.DataFrame(test_result_complete)
test_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Results.csv', index=False)
test_df_complete(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Results Complete.csv', index=False)

In [25]:
result1_df = pd.DataFrame(result1)
result1_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\First Batch\First Batch Experiment 3.csv')

In [27]:
result1_df

,breast-cancer-wisconsin.csv,HTRU_2.csv,Indian Liver Patient Dataset (ILPD).csv,magic04.csv,monks.csv,NHANES_age_prediction.csv,Phising Data.csv,Raisin_Dataset.csv,Rice Cammeo Osmancik.csv,transfusion.csv
0,0.610251,0.937979,0.534059,0.675552,0.552477,0.843719,0.7232,0.475556,0.559843,0.739387
1,0.910635,0.974299,0.663188,0.794953,0.619281,0.938533,0.8968,0.644444,0.586352,0.716793
2,0.689301,0.924236,0.668603,0.667560,0.545384,0.813455,0.6352,0.474444,0.560892,0.726054
3,0.802195,0.963349,0.715154,0.741483,0.567110,0.844157,0.8152,0.496667,0.572178,0.762054
4,0.702920,0.952172,0.715154,0.751209,0.592893,0.884102,0.8984,0.477778,0.572178,0.762054
5,0.928282,0.975640,0.680369,0.802787,0.622810,0.937240,0.9184,0.484444,0.511811,0.697694
6,0.687937,0.913900,0.625499,0.661094,0.549525,0.850298,0.6216,0.462222,0.572178,0.762054
7,0.737724,0.935468,0.715154,0.681651,0.557754,0.844157,0.6728,0.464444,0.557743,0.762054
8,0.683802,0.922060,0.715154,0.656940,0.549525,0.840210,0.6400,0.462222,0.572178,0.762054
9,0.710230,0.928034,0.715154,0.682597,0.558937,0.870135,0.7264,0.466667,0.572178,0.762054


In [26]:
result1_complete

{'breast-cancer-wisconsin.csv': {0: np.float64(0.6102514907717704),
  1: np.float64(0.910635131597519),
  2: np.float64(0.6893009424209595),
  3: np.float64(0.8021952211856842),
  4: np.float64(0.7029198706150055),
  5: np.float64(0.9282821834087371),
  6: np.float64(0.6879369139671325),
  7: np.float64(0.7377237915992737),
  8: np.float64(0.6838022172451019),
  9: np.float64(0.7102301836013794),
  10: np.float64(0.777344411611557),
  11: np.float64(0.81436488032341),
  12: np.float64(0.8561168015003204),
  13: np.float64(0.9473572075366974)},
 'HTRU_2.csv': {0: np.float64(0.9379785895347595),
  1: np.float64(0.9742989361286163),
  2: np.float64(0.9242355704307557),
  3: np.float64(0.9633491694927215),
  4: np.float64(0.9521722555160522),
  5: np.float64(0.9756396472454071),
  6: np.float64(0.9139003753662109),
  7: np.float64(0.9354684054851532),
  8: np.float64(0.9220604538917542),
  9: np.float64(0.9280344486236572),
  10: np.float64(0.9664218008518219),
  11: np.float64(0.952618229

In [9]:
result2, result2_complete = hp_space_experiment_from_split(r'D:\MATTHEW\UROPS\Datasets\Second Batch', hp_set)

banana_quality.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 422us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 425us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 512us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 417us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 430us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 684us/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 488us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 475us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 486us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 516us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/2

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 461us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 454us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 522us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 978us/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 517us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 559us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 943us/step
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 508us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step
2

...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s

C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1000us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
7/7 ━━━━━━━━━━━━━━━━━

In [10]:
result2_df = pd.DataFrame(result2)
result2_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Second Batch\Second Batch Experiment 3.csv')

In [15]:
result1_complete_df = pd.DataFrame(result1_complete)
result2_complete_df = pd.DataFrame(result2_complete)

In [32]:
result1_complete_df = result1_complete_df.T
result2_complete_df = result2_complete_df.T

In [16]:
result1_complete_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\First Batch\First Batch Experiment 3 Validation Results.csv')
result2_complete_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Second Batch\Second Batch Experiment 3 Validation Results.csv')

In [2]:
result2_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Second Batch\Second Batch Experiment 3.csv', index_col=0)
result1_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\First Batch\First Batch Experiment 3.csv', index_col=0)

In [3]:
result2_complete_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Second Batch\Second Batch Experiment 3 Validation Results.csv', index_col=0)
result1_complete_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\First Batch\First Batch Experiment 3 Validation Results.csv', index_col=0)

In [4]:
result1_df = result1_df.T
result2_df = result2_df.T
result1_complete_df = result1_complete_df.T
result2_complete_df = result2_complete_df.T

In [2]:
def importance_encoding(df, id_max):
    # print(df)
    alpha = 0.05
    # print(id_max)
    result = {}
    for index, row in df.iterrows():
        # print(index)
        cur_result = {}
        for col, values in row.items():
            # print(row.iloc[id_max[index]])
            # print(values)
            if type(values) == str and type(row.iloc[int(id_max[index])]) == str:
                t_stat, p_value = stats.ttest_rel(ast.literal_eval(row.iloc[int(id_max[index])]), ast.literal_eval(values))
            elif type(row.iloc[int(id_max[index])]) == str:
                t_stat, p_value = stats.ttest_rel(ast.literal_eval(row.iloc[int(id_max[index])]), values)
            elif type(values) == str:
                t_stat, p_value = stats.ttest_rel(row.iloc[int(id_max[index])], ast.literal_eval(values))
            else:
                t_stat, p_value = stats.ttest_rel(row.iloc[int(id_max[index])], values)
            if p_value < alpha:
                cur_result[col] = 0
            else:
                cur_result[col] = 1
        result[index] = cur_result
    return pd.DataFrame(result).T


In [15]:
all_encoded_df = pd.DataFrame()
all_result_df = pd.DataFrame()
all_result_complete_df = pd.DataFrame()
for i in range(1, 11):
    print(f'Batch {i}')
    dir = r'D:\MATTHEW\UROPS\Datasets\All Datasets\Batch ' + str(i)
    df = pd.read_csv(dir + '\\' + f'Batch {i} Results.csv', index_col=0)
    df_complete = pd.read_csv(dir + '\\' + f'Batch {i} Results Complete.csv', index_col=0)
    encoded_df = importance_encoding(df_complete, dict(df.idxmax(axis=1)))
    all_encoded_df = pd.concat([all_encoded_df, encoded_df])
    all_result_df = pd.concat([all_result_df, df])
    all_result_complete_df = pd.concat([all_result_complete_df, df_complete])
    

Batch 1
Batch 2
Batch 3
Batch 4
Batch 5
Batch 6
Batch 7
Batch 8
Batch 9
Batch 10


In [ ]:
all_result_complete_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Results Complete.csv')
all_result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\All Datasets\All Datasets Results.csv')

: 

In [19]:
result1_complete_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
breast-cancer-wisconsin.csv,"[0.7246376872062683, 0.695652186870575, 0.6811...","[0.8985507488250732, 0.8985507488250732, 0.869...","[0.6231883764266968, 0.7101449370384216, 0.681...","[0.8985507488250732, 0.9855072498321533, 0.840...","[0.9130434989929199, 0.7101449370384216, 0.855...","[0.9420289993286133, 0.9130434989929199, 0.913...","[0.6521739363670349, 0.7101449370384216, 0.681...","[0.6231883764266968, 0.7101449370384216, 0.681...","[0.6231883764266968, 0.7101449370384216, 0.681...","[0.6231883764266968, 0.7101449370384216, 0.681...","[0.9420289993286133, 0.95652174949646, 0.68115...","[0.95652174949646, 0.7101449370384216, 0.88405...","[0.6231883764266968, 0.7101449370384216, 0.898...","[0.9710144996643066, 0.95652174949646, 0.92753..."
HTRU_2.csv,"[0.9558659195899963, 0.9089385271072388, 0.901...","[0.9776536226272583, 0.9759776592254639, 0.966...","[0.9173184633255005, 0.9089385271072388, 0.970...","[0.9743016958236694, 0.9089385271072388, 0.970...","[0.9787709712982178, 0.9810056090354919, 0.967...","[0.9681563973426819, 0.9810056090354919, 0.967...","[0.9173184633255005, 0.9089385271072388, 0.901...","[0.9776536226272583, 0.978212296962738, 0.9692...","[0.9173184633255005, 0.9793295860290527, 0.901...","[0.9173184633255005, 0.9675977826118469, 0.901...","[0.9754189848899841, 0.9765363335609436, 0.901...","[0.9804469347000122, 0.9804469347000122, 0.968...","[0.9173184633255005, 0.9793295860290527, 0.967...","[0.9798882603645325, 0.9804469347000122, 0.968..."
Indian Liver Patient Dataset (ILPD).csv,"[0.6379310488700867, 0.3103448152542114, 0.620...","[0.5862069129943848, 0.6896551847457886, 0.637...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.568965494632721, 0.6551724076271057, 0.6724...","[0.6379310488700867, 0.4655172526836395, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6379310488700867, 0.6896551847457886, 0.672...","[0.6206896305084229, 0.7068965435028076, 0.672..."
magic04.csv,"[0.6393270492553711, 0.6372239589691162, 0.661...","[0.7870662212371826, 0.8170347213745117, 0.779...","[0.6414300799369812, 0.7644584774971008, 0.652...","[0.6493164896965027, 0.7733964323997498, 0.760...","[0.6393270492553711, 0.6372239589691162, 0.695...","[0.7776025533676147, 0.827549934387207, 0.7970...","[0.6393270492553711, 0.6372239589691162, 0.652...","[0.7581493258476257, 0.7171398401260376, 0.652...","[0.6393270492553711, 0.6372239589691162, 0.652...","[0.6393270492553711, 0.812302827835083, 0.6524...","[0.707676112651825, 0.7754994630813599, 0.7728...","[0.6393270492553711, 0.8201892971992493, 0.652...","[0.7907465696334839, 0.8159831762313843, 0.804...","[0.7981072664260864, 0.8191377520561218, 0.794..."
monks.csv,"[0.5730994343757629, 0.5497075915336609, 0.502...","[0.6549707651138306, 0.6666666865348816, 0.625...","[0.567251443862915, 0.4912280738353729, 0.5380...","[0.5730994343757629, 0.5497075915336609, 0.538...","[0.6549707651138306, 0.6491228342056274, 0.637...","[0.6432748436927795, 0.6842105388641357, 0.608...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.567251443862915, 0.5497075915336609, 0.5380...","[0.6257309913635254, 0.6491228342056274, 0.619...","[0.6432748436927795, 0.6725146174430847, 0.614..."
NHANES_age_prediction.csv,"[0.7938596606254578, 0.8552631735801697, 0.907...","[0.9298245906829834, 0.8991228342056274, 0.942...","[0.8201754093170166, 0.8552631735801697, 0.833...","[0.8201754093170166, 0.8552631735801697, 0.

In [6]:
encoded_df1 = importance_encoding(result1_complete_df, dict(result1_df.idxmax(axis=1)))
encoded_df2 = importance_encoding(result2_complete_df, dict(result2_df.idxmax(axis=1)))

In [12]:
encoded_df1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
breast-cancer-wisconsin.csv,0,0,0,0,0,0,0,0,0,0,0,0,0,1
HTRU_2.csv,0,1,0,1,1,1,0,1,0,0,1,1,0,1
Indian Liver Patient Dataset (ILPD).csv,0,1,1,1,1,0,1,1,1,1,1,1,1,1
magic04.csv,0,1,0,0,0,1,0,0,0,0,0,0,1,1
monks.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1
NHANES_age_prediction.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1
Phising Data.csv,0,1,0,0,1,1,0,0,0,0,0,1,1,1
Raisin_Dataset.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1
Rice Cammeo Osmancik.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1
transfusion.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [10]:
test_encoded_df = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\Experiment 4\Experiment 4 Encoded Results.csv", index_col=0)

In [11]:
test_encoded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
breast-cancer-wisconsin.csv,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
HTRU_2.csv,0,1,0,1,1,1,0,1,0,0,1,1,0,1,1
Indian Liver Patient Dataset (ILPD).csv,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1
magic04.csv,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1
monks.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
NHANES_age_prediction.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
Phising Data.csv,0,1,0,0,1,1,0,0,0,0,0,1,1,1,1
Raisin_Dataset.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
Rice Cammeo Osmancik.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
transfusion.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [17]:
def binary_dist(x, y):
    dist = 0
    for i in range(len(x)):
        dist += abs(x[i] - y[i])
    return dist


In [28]:
def find_hp_dist(hp1, hp2):
    return np.linalg.norm(np.array(hp1[1])-np.array(hp2[1]))

In [7]:
def find_dof(hp, input_dim = 10):
    dof = 0
    for i in range(hp[0]):
        cur_dim = hp[1][i]
        if i == 0:
            prev_dim = input_dim
        else:
            prev_dim = hp[1][i - 1]
        dof += cur_dim + cur_dim * prev_dim
    dof += 1 + hp[1][-1]
    return dof


In [2]:
def find_input_dim(file, dir):
    df = pd.read_csv(dir + '\\' + file)
    return len(df.columns) - 1

In [4]:
def find_dof_dist(dof1, dof2):
    return abs(dof1 - dof2)

In [6]:
def find_dof_midpoint(dof1, dof2, input_dim = 10):
    min_dof = min(dof1, dof2)
    max_dof = max(dof1, dof2)
    for i in range(1, 17):
        for j in range(0, 17):
            if j > 0:
                for k in range(0, 17):
                    if k == 0:
                        cur_dof = input_dim * i + i* j + j
                    else:
                        cur_dof = input_dim * i + i* j + j * k + k
                    if cur_dof > min_dof and cur_dof < max_dof:
                        cur_hp = [i, j, k]
                        layers = 0
                        for n in cur_hp:
                            if n > 0:
                                layers += 1
                        return (layers, cur_hp)
            else:
                cur_dof = input_dim * i + i* j + j
                if cur_dof > min_dof and cur_dof < max_dof:
                        cur_hp = [i, j, 0]
                        layers = 0
                        for n in cur_hp:
                            if n > 0:
                                layers += 1
                        return (layers, cur_hp)

                    

In [27]:
def find_hp_midpoint(hp1, hp2):
    neurons = (np.array(hp1[1]) + np.array(hp2[1])) // 2
    layers = 0
    for i in neurons:
        if i != 0:
            layers += 1
    return (layers, neurons.tolist())

In [5]:
def update_encoding(dir, df, df_complete, new_hp, hp_set):
    id_max = df.idxmax(axis = 1)
    new_df, new_df_complete = hp_space_experiment_from_split(dir, new_hp)
    new_df = pd.DataFrame(new_df).T
    new_df_complete = pd.DataFrame(new_df_complete).T
    # print(new_df)
    # print(new_df_complete)
    for col_id, col in new_df.items():
        df[str(int(df.columns[-1]) + 1)] = col
    for col_id, col in new_df_complete.items():
        df_complete[str(int(df_complete.columns[-1]) + 1)] = col
    hp_set += new_hp
    return df, df_complete, importance_encoding(df_complete, dict(df.idxmax(axis=1))), hp_set

In [47]:
def find_new_hp(encoded_df, hp_set, cur_col_id):
    new_hp = []
    cur_col = encoded_df.iloc[:, int(cur_col_id)]
    nn_id = 0
    nn_dist = float('inf')
    for col_id, col in encoded_df.items():
        cur_dist = find_hp_dist(hp_set[int(col_id)], hp_set[int(cur_col_id)])
        # print(col_id, type(col_id))
        # print(cur_col_id, type(cur_col_id))
        if int(col_id) != int(cur_col_id) and  cur_dist < nn_dist:
            nn_id = col_id
            nn_dist = cur_dist
        # print(nn_id)
        # print(nn_dist)
    print(f'NN_ID: {nn_id}')
    print(binary_dist(cur_col.values, encoded_df.iloc[:, int(nn_id)].values))
    if binary_dist(cur_col.values, encoded_df.iloc[:, int(nn_id)].values) >= len(cur_col.values) // 2:
        cur_hp = find_dof_midpoint(find_dof(hp_set[int(nn_id)]), find_dof(hp_set[int(cur_col_id)]))
        if cur_hp not in hp_set:
            new_hp.append(cur_hp)
    print(new_hp)
    return new_hp

In [49]:
def find_new_hp_dof(encoded_df, hp_set, cur_col_id):
    new_hp = []
    cur_col = encoded_df.iloc[:, int(cur_col_id)]
    nn_id = 0
    nn_dist = float('inf')
    for col_id, col in encoded_df.items():
        cur_dist = find_dof_dist(find_dof(hp_set[int(col_id)]), find_dof(hp_set[int(cur_col_id)]))
        if int(col_id) != int(cur_col_id) and  cur_dist < nn_dist:
            nn_id = col_id
            nn_dist = cur_dist
        # print(nn_id)
        # print(nn_dist)
    print(nn_id)
    print(binary_dist(cur_col.values, encoded_df.iloc[:, int(nn_id)].values))
    if binary_dist(cur_col.values, encoded_df.iloc[:, int(nn_id)].values) >= len(cur_col.values) // 2:
        cur_hp = find_hp_midpoint(hp_set[int(nn_id)], hp_set[int(cur_col_id)])
        if cur_hp not in hp_set:
            new_hp.append(cur_hp)
    print(new_hp)
    return new_hp

In [24]:
def experiment3_dof(df, df_complete, encoded_df, dir, hp_set):
    col_id = 0
    while col_id < len(encoded_df.columns):
        print(col_id)
        new_hp = find_new_hp_dof(encoded_df, hp_set, col_id)
        if new_hp == []:
            col_id += 1
            continue
        # print(new_hp)
        df, df_complete, encoded_df, hp_set = update_encoding(dir, df, df_complete, new_hp, hp_set)
    return df, df_complete, encoded_df, hp_set


In [25]:
def experiment3(df, df_complete, encoded_df, dir, hp_set):
    col_id = 0
    while col_id < len(encoded_df.columns):
        print(col_id)
        new_hp = find_new_hp(encoded_df, hp_set, col_id)
        if new_hp == []:
            col_id += 1
            continue
        # print(new_hp)
        df, df_complete, encoded_df, hp_set = update_encoding(dir, df, df_complete, new_hp, hp_set)
    return df, df_complete, encoded_df, hp_set

In [112]:
hp_set

[(1, [1, 0, 0]),
 (1, [32, 0, 0]),
 (2, [1, 1, 0]),
 (2, [1, 32, 0]),
 (2, [32, 1, 0]),
 (2, [32, 32, 0]),
 (3, [1, 1, 1]),
 (3, [1, 1, 32]),
 (3, [1, 32, 1]),
 (3, [32, 1, 1]),
 (3, [1, 32, 32]),
 (3, [32, 32, 1]),
 (3, [32, 1, 32]),
 (3, [32, 32, 32])]

In [2]:
test_hp_set = [(1, [1, 0, 0]),
 (1, [16, 0, 0]),
 (2, [1, 1, 0]),
 (2, [1, 16, 0]),
 (2, [16, 1, 0]),
 (2, [16, 16, 0]),
 (3, [1, 1, 1]),
 (3, [1, 1, 16]),
 (3, [1, 16, 1]),
 (3, [16, 1, 1]),
 (3, [1, 16, 16]),
 (3, [16, 16, 1]),
 (3, [16, 1, 16]),
 (3, [16, 16, 16])]

In [56]:
result1 = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Batch 1\Batch 1 Results.csv", index_col=0)
result1_complete = pd.read_csv(r"D:\MATTHEW\UROPS\Datasets\All Datasets\Batch 1\Batch 1 Results Complete.csv", index_col = 0)
encoded1 = importance_encoding(result1_complete, dict(result1.idxmax(axis=1)))

In [57]:
encoded1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
100_plants_margin.csv,0,1,0,0,0,1,0,0,0,0,0,1,0,0
100_plants_shape.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1
100_plants_texture.csv,0,1,0,0,1,1,0,0,0,0,0,1,0,1
amazon_review.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1
analcatdata_authorship.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1
AP_Colon_Kidney.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1
ar4.csv,1,1,0,1,1,1,0,0,1,1,0,0,1,1
ar6.csv,1,1,1,1,1,1,0,1,1,1,1,1,1,0
banana_quality.csv,0,0,0,0,0,1,0,0,0,0,0,1,0,1
Banknote.csv,0,0,0,0,1,1,0,0,0,0,0,1,1,1


In [58]:
test_result_df, test_result_complete_df, test_encoded_df, test_hp_set = experiment3_dof(result1, result1_complete, encoded1, r'D:\MATTHEW\UROPS\Datasets\All Datasets\Batch 1', test_hp_set)

0
2
1
[]
1
4
4
[]
2
0
1
[]
3
7
1
[]
4
9
2
[]
5
11
3
[]
6
2
1
[]
7
8
1
[]
8
7
1
[]
9
4
2
[]
10
12
2
[]
11
5
3
[]
12
9
1
[]
13
11
5
[(3, [16, 16, 8])]
100_plants_margin.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 886us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 887us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


...

35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
{'100_plants_margin.csv': {0: [0.4749999940395355, 0.5562499761581421, 0.4749999940395355, 0.5562499761581421, 0.5625, 0.606249988079071, 0.5562499761581421, 0.5687500238418579, 0.53125, 0.53125]}, '100_plants_shape.csv': {0: [0.4749999940395355, 0.48750001192092896, 0.48124998807907104, 0.4437499940395355, 0.42500001192092896, 0.48750001192092896, 0.512499988079071, 0.4937500059604645, 0.4749999940395355, 0.4937500059604645]}, '100_plants_texture.csv': {0: [0.5874999761581421, 0.574999988079071, 0.543749988079071, 0.543749988079071, 0.5625, 0.5249999761581421, 0.5562499761581421, 0.612500011920929, 0.5345911979675293, 0.5911949872970581]}, 'amazon_review.csv': {0: [0.40666666626930237, 0.47333332896232605, 0.4933333396911621, 0.4933333396911621, 0.4533333480358124, 0.5600000023841858, 0.46666666865348816, 0.5533333420753479, 0.4866666793823242, 0.46000000834465027]}, 'analcatdata_authorship.csv': {0: [0.76056

In [60]:
test_encoded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
100_plants_margin.csv,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1
100_plants_shape.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
100_plants_texture.csv,0,1,0,0,1,1,0,0,0,0,0,1,0,1,1
amazon_review.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
analcatdata_authorship.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
AP_Colon_Kidney.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
ar4.csv,1,1,0,1,1,1,0,0,1,1,0,0,1,1,1
ar6.csv,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1
banana_quality.csv,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1
Banknote.csv,0,0,0,0,1,1,0,0,0,0,0,1,1,1,1


In [ ]:
test_encoded_df = pd.DataFrame(importance_encoding(test_result_complete_df, test_result_df.idxmax(axis=1)))


In [14]:
test_encoded_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Encoded Results.csv')
test_result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Results.csv')
test_result_complete_df.to_csv((r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Complete Results.csv'))

NameError: name 'test_encoded_df' is not defined

In [169]:
test_result_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Results.csv', index_col=0)
test_result_complete_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Complete Results.csv', index_col=0)
test_encoded_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Test Batch\Experiment 3 Encoded Results.csv', index_col=0)

In [170]:
test_encoded_df = test_encoded_df.T

In [171]:
test_encoded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
banana_quality.csv,0,0,0,0,0,1,0,0,0,0,0,1,1,1
car_data.csv,1,0,1,1,1,1,1,1,1,1,1,1,1,1
Healthcare-Diabetes.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1


In [22]:
test_result_df, test_result_complete_df, test_encoded_df, test_hp_set = experiment3(pd.concat([result1_df, result2_df]), pd.concat([result1_complete_df, result2_complete_df]), pd.concat([encoded_df1, encoded_df2]), r'D:\MATTHEW\UROPS\Datasets\Experiment 4', test_hp_set)

0
2
1
[]
1
4
12
[]
2
0
1
[]
3
8
2
[]
4
1
12
[]
5
11
12
[]
6
2
0
[]
7
6
1
[]
8
3
2
[]
9
4
3
[]
10
7
0
[]
11
5
12
[]
12
7
4
[]
13
10
15
[(3, [8, 16, 16])]
banana_quality.csv


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 531us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 991us/step


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 560us/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\Matthew Wijaya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
{'banana_quality.csv': {0: [0.9737499952316284, 0.9700000286102295, 0.9737499952316284, 0.9574999809265137, 0.9712499976158142, 0.9649999737739563, 0.9662500023841858, 0.9737499952316284, 0.9725000262260437, 0.9637500047683716]}, 'breast-cancer-wisconsin.csv': {0: [0.9275362491607666, 0.9420289993286133, 0.9130434989929199, 0.9558823704719543, 0.9264705777168274, 0.9558823704719543, 0.8676470518112183, 0.9558823704719543, 0.9558823704719543, 0.9117646813392639]}, 'car_data.csv': {0: [0.6399999856948853, 0.3499999940395355, 0.4000000059604645, 0.5400000214576721, 0.550000011920929, 0.4000000059604645, 0.5699999928474426, 0.4099999964237213, 0.6600000262260437, 0.41999998688697815]}, 'Healthcare-Diabetes.csv': {0: [0.7220216393470764, 0.7581227421760559, 0.6570397019386292, 0.7364621162414551, 0.7328519821166992, 0.6570397019386292, 0.6534296274185181, 0.6859205961227417, 0.7246376872062683, 0.6811594367027283

In [23]:
test_encoded_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Experiment 4\Experiment 4 Encoded Results.csv')
test_result_complete_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Experiment 4\Experiment 4 Complete Results.csv')
test_result_df.to_csv(r'D:\MATTHEW\UROPS\Datasets\Experiment 4\Experiment 4 Results.csv')

In [24]:
test_hp_set

[(1, [1, 0, 0]),
 (1, [16, 0, 0]),
 (2, [1, 1, 0]),
 (2, [1, 16, 0]),
 (2, [16, 1, 0]),
 (2, [16, 16, 0]),
 (3, [1, 1, 1]),
 (3, [1, 1, 16]),
 (3, [1, 16, 1]),
 (3, [16, 1, 1]),
 (3, [1, 16, 16]),
 (3, [16, 16, 1]),
 (3, [16, 1, 16]),
 (3, [16, 16, 16]),
 (3, [8, 16, 16])]

In [3]:
test_encoded_df = pd.read_csv(r'D:\MATTHEW\UROPS\Datasets\Combined Batch\Experiment 3 Encoded Results.csv', index_col=0)

In [26]:
test_encoded_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
breast-cancer-wisconsin.csv,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
HTRU_2.csv,0,1,0,1,1,1,0,1,0,0,1,1,0,1,1
Indian Liver Patient Dataset (ILPD).csv,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1
magic04.csv,0,1,0,0,0,1,0,0,0,0,0,0,1,1,1
monks.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
NHANES_age_prediction.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1
Phising Data.csv,0,1,0,0,1,1,0,0,0,0,0,1,1,1,1
Raisin_Dataset.csv,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0
Rice Cammeo Osmancik.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
transfusion.csv,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [27]:
test_result_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
breast-cancer-wisconsin.csv,0.610251,0.910635,0.689301,0.802195,0.702920,0.928282,0.687937,0.737724,0.683802,0.710230,0.777344,0.814365,0.856117,0.947357,0.931202
HTRU_2.csv,0.937979,0.974299,0.924236,0.963349,0.952172,0.975640,0.913900,0.935468,0.922060,0.928034,0.966422,0.952618,0.967260,0.975360,0.974187
Indian Liver Patient Dataset (ILPD).csv,0.534059,0.663188,0.668603,0.715154,0.715154,0.680369,0.625499,0.715154,0.715154,0.715154,0.715154,0.715154,0.715154,0.701210,0.675529
magic04.csv,0.675552,0.794953,0.667560,0.741483,0.751209,0.802787,0.661094,0.681651,0.656940,0.682597,0.744795,0.685068,0.776236,0.804259,0.799895
monks.csv,0.552477,0.619281,0.545384,0.567110,0.592893,0.622810,0.549525,0.557754,0.549525,0.558937,0.578277,0.608793,0.594080,0.633364,0.623977
NHANES_age_prediction.csv,0.843719,0.938533,0.813455,0.844157,0.884102,0.937240,0.850298,0.844157,0.840210,0.870135,0.859947,0.867841,0.893779,0.936351,0.939862
Phising Data.csv,0.723200,0.896800,0.635200,0.815200,0.898400,0.918400,0.621600,0.672800,0.640000,0.726400,0.841600,0.882400,0.876800,0.917600,0.905600
Raisin_Dataset.csv,0.475556,0.644444,0.474444,0.496667,0.477778,0.484444,0.462222,0.464444,0.462222,0.466667,0.462222,0.462222,0.475556,0.496667,0.526667
Rice Cammeo Osmancik.csv,0.559843,0.586352,0.560892,0.572178,0.572178,0.511811,0.572178,0.557743,0.572178,0.572178,0.527297,0.572178,0.614698,0.569029,0.567717
transfusion.csv,0.739387,0.716793,0.726054,0.762054,0.762054,0.697694,0.762054,0.762054,0.762054,0.762054,0.750054,0.762054,0.760721,0.686883,0.740721
